In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Framework

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'framework-container'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

In [59]:
import json
import os
docker_mages_directory = "/home/ec2-user/SageMaker/docker"
docker_settings_file = "/etc/docker/daemon.json"

!sudo chmod 777  /etc/docker/daemon.json

if not os.path.exists(docker_mages_directory):
    os.mkdir(docker_mages_directory)
with open(docker_settings_file, "r") as f:
    docker_settings = json.loads(f.read())

docker_settings["data-root"] = docker_mages_directory

with open(docker_settings_file, "w") as f:
    f.write(json.dumps(docker_settings))
    
!sudo service docker restart

Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]


In [70]:
# build and push training image
# !chmod u+x ./build_and_push.sh 
!./build_and_push.sh autogluon containers/training/Dockerfile.gpu

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  40.45MB
Step 1/13 : ARG CUDA_VER=11.0
Step 2/13 : ARG LINUX_VER=ubuntu18.04
Step 3/13 : ARG PYTHON_VER=3.7
Step 4/13 : ARG RAPIDS_VER=0.19
Step 5/13 : ARG FROM_IMAGE=rapidsai/rapidsai-core
Step 6/13 : FROM ${FROM_IMAGE}:${RAPIDS_VER}-cuda${CUDA_VER}-runtime-${LINUX_VER}-py${PYTHON_VER}
 ---> 5919023f2ac9
Step 7/13 : ARG CUDA_VER
 ---> Using cache
 ---> d3db16ca4e8b
Step 8/13 : ENV PYTHONDONTWRITEBYTECODE=1     PYTHONUNBUFFERED=1     PYTHONIOENCODING=UTF-8     LANG=C.UTF-8     LC_ALL=C.UTF-8     SAGEMAKER_TRAINING_MODULE=sagemaker_mxnet_container.training:main     LD_LIBRARY_PATH=/opt/conda/envs/rapids/lib:/usr/local/lib:$LD_LIBRARY_PATH     PATH=/opt/conda/envs/rapids/bin:/usr/local/bin:$PATH
 ---> Using cach

In [ ]:
# !./build_and_push.sh autogluon-small containers/training/Dockerfile_smaller.gpu

In [50]:
image_uri = "152804913371.dkr.ecr.us-east-1.amazonaws.com/autogluon:latest"

In [83]:
class AutoGluon(Framework):
    def __init__(
        self,
        entry_point,
        source_dir=None,
        hyperparameters=None,
        py_version="py3",
        framework_version=None,
        image_uri=None,
        distributions=None,
        **kwargs
    ):
        super().__init__(
            entry_point, source_dir, hyperparameters, image_uri=image_uri, **kwargs
        )
    
    def _configure_distribution(self, distributions):
        return
    
    def create_model(
        self,
        model_server_workers=None,
        role=None,
        vpc_config_override=None,
        entry_point=None,
        source_dir=None,
        dependencies=None,
        image_name=None,
        **kwargs
    ):
        return None

In [84]:
tabular_estimator =  AutoGluon(image_uri=image_uri,
                      role=role,
                      entry_point="training_scripts/tabular.py",
#                       source_dir='source_dir/',
                      instance_count=1, 
                      #instance_type="local", # we use local mode
                      instance_type="ml.p3.2xlarge",
                      base_job_name="tabular-train",
                      )

In [85]:
tabular_estimator.fit()

2021-05-05 17:04:07 Starting - Starting the training job...
2021-05-05 17:04:31 Starting - Launching requested ML instancesProfilerReport-1620234246: InProgress
......
2021-05-05 17:05:32 Starting - Preparing the instances for training.........
2021-05-05 17:06:58 Downloading - Downloading input data
2021-05-05 17:06:58 Training - Downloading the training image...............................2021-05-05 17:12:12,077 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-05-05 17:12:12,100 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': 'tabular.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_CHANNELS': '[]', 'SM_CURRENT_HOST': 'algo-1', 'SM_MODULE_NAME': 'tabu

In [71]:
from sagemaker.inputs import TrainingInput

In [73]:
key_prefix="otto_group"

In [75]:
train_input = TrainingInput(f"s3://{bucket}/{key_prefix}/train.csv", content_type="text/csv")
validation_input = TrainingInput(f"s3://{bucket}/{key_prefix}/test.csv", content_type="text/csv")

In [79]:
rapids_estimator =  AutoGluon(image_uri=image_uri,
                      role=role,
                      entry_point="training_scripts/rapids.py",
#                       source_dir='source_dir/',
                      instance_count=1, 
                      #instance_type="local", # we use local mode
                      instance_type="ml.p3.2xlarge",
                      base_job_name="rapids-train",
                      )

In [80]:
rapids_estimator.fit({'train': train_input, 'test': validation_input}, 
              wait=True)

2021-05-05 14:54:11 Starting - Starting the training job...
2021-05-05 14:54:36 Starting - Launching requested ML instancesProfilerReport-1620226451: InProgress
......
2021-05-05 14:55:36 Starting - Preparing the instances for training.........
2021-05-05 14:57:01 Downloading - Downloading input data
2021-05-05 14:57:01 Training - Downloading the training image..............................
2021-05-05 15:02:12 Training - Training image download completed. Training in progress..2021-05-05 15:02:13,076 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-05-05 15:02:13,099 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': 'rapids.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{"test":{"ContentType":"text/csv","R